In [1]:
import cv2 
import numpy as np

In [2]:
import os

In [3]:
import json
with open('./tcdata/tile_round1_train_20201231/train_annos.json', 'r') as fp:
     results=json.load(fp)

In [4]:
dict_data=results[1]

In [5]:
dict_data

{'name': '235_2_t20201127123021723_CAM2.jpg',
 'image_height': 6000,
 'image_width': 8192,
 'category': 5,
 'bbox': [1876.06, 998.04, 1883.06, 1004.04]}

#### Edit get_sliced_yolo_box()
```python
save_path='./tile_round1_train_20201231/train_imgs_sliced/'
target_size=100
expand_ratio=1.5
im = cv2.imread('./tile_round1_train_20201231/train_imgs/'+dict_data['name'])
fname=dict_data['name'].split('.j')[0]

hy,wx,_=im.shape
gap_y=hy%target_size//2
gap_x=wx%target_size//2

tag_xy=dict_data['bbox']

new_x0=((round(tag_xy[0])-gap_x)//target_size)*target_size+gap_x
new_y0=((round(tag_xy[1])-gap_y)//target_size)*target_size+gap_y

w=round(target_size*expand_ratio)
h=round(target_size*expand_ratio)


out_x=(np.mean([tag_xy[0],tag_xy[2]])-new_x0)/w
out_y=(np.mean([tag_xy[1],tag_xy[3]])-new_y0)/h
out_w=(tag_xy[2]-tag_xy[0])/w
out_h=(tag_xy[3]-tag_xy[1])/h
out_string="{} {:.6f} {:.6f} {:.6f} {:.6f}\n".format(dict_data['category'],out_x,out_y,out_w,out_h)

if hy>=new_y0+h and wx>=new_x0+w:
    new_img=im[new_y0:new_y0+h,new_x0:new_x0+w,:]
else:
    new_img=im[new_y0:hy,new_x0:wx,:]
    new_img=np.pad(new_img, ((0,new_y0+h-hy),(0,new_x0+w-wx),(0,0)), 'constant', constant_values=0)

pic_fname=fname+'_'+str(new_x0)+'_'+str(new_y0)+'.jpg'
if os.path.exists(pic_fname):
    pass
else:
    cv2.imwrite(save_path+pic_fname, new_img,[int(cv2.IMWRITE_JPEG_QUALITY), 100])

anno_fname=fname+'_'+str(new_x0)+'_'+str(new_y0)+'.txt'
with open(save_path+anno_fname,'a+') as f:
    f.write(out_string)
```

In [6]:
def get_sliced_yolo_box(dict_data):
    #parameters
    pic_save_path='./tcdata/train_imgs_sliced_160/images/'
    anno_save_path='./tcdata/train_imgs_sliced_160/labels/'
    target_size=100
    expand_ratio=1.6 #2.24
    
    im = cv2.imread('./tcdata/tile_round1_train_20201231/train_imgs/'+dict_data['name'])
    fname=dict_data['name'].split('.j')[0]

    #hight_y(hy),width_x(wx)
    hy,wx,_=im.shape
    gap_y=hy%target_size//2
    gap_x=wx%target_size//2

    tag_xy=dict_data['bbox']

    new_x0=((round(tag_xy[0])-gap_x)//target_size)*target_size+gap_x
    new_y0=((round(tag_xy[1])-gap_y)//target_size)*target_size+gap_y

    #new img size
    w=round(target_size*expand_ratio)
    h=round(target_size*expand_ratio)
    
    #new boundary
    new_x1=new_x0+w
    new_y1=new_y0+h


    out_x=(np.mean([tag_xy[0],min(tag_xy[2],new_x1)])-new_x0)/w
    out_y=(np.mean([tag_xy[1],min(tag_xy[3],new_y1)])-new_y0)/h
    out_w=(min(tag_xy[2],new_x1)-tag_xy[0])/w
    out_w=min(1,out_w)
    out_h=(min(tag_xy[3],new_y1)-tag_xy[1])/h
    out_h=min(1,out_h)
    
    out_string="{} {:.6f} {:.6f} {:.6f} {:.6f}\n".format(dict_data['category'],out_x,out_y,out_w,out_h)

    if hy>=new_y1 and wx>=new_x1:
        new_img=im[new_y0:new_y1,new_x0:new_x1,:]
    else:
        new_img=im[new_y0:hy,new_x0:wx,:]
        new_img=np.pad(new_img, ((0,new_y1-hy),(0,new_x1-wx),(0,0)), 'constant', constant_values=0)

    pic_fname=fname+'_'+str(wx)+'_'+str(hy)+'_'+str(new_x0)+'_'+str(new_y0)+'.jpg'
    if os.path.exists(pic_fname):
        pass
    else:
        cv2.imwrite(pic_save_path+pic_fname, new_img,[int(cv2.IMWRITE_JPEG_QUALITY), 100])

    anno_fname=fname+'_'+str(wx)+'_'+str(hy)+'_'+str(new_x0)+'_'+str(new_y0)+'.txt'
    with open(anno_save_path+anno_fname,'a+') as f:
        f.write(out_string)

In [7]:
for dict_data in results:
    get_sliced_yolo_box(dict_data)